# Fase 1: Extracción de Datos de API de MoviesDataset

In [9]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [18]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[]}

# creamos una lista para almacenar las respuestas de la funcion de llamada a la API
lista_respuestas = []


#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

# API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
# headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

titleType_list = ['movie', 'short']
genere_list = ['Drama', 'Action', 'Comedy']

# querystring = {"titleType":"short","startYear":"2010","endYear":"2024","genre":"Drama"}


In [23]:
# Definimos una fucnción para extraer información de la API y almacenarla en un diccionario

def llamar_API (url,type='movie', genere='Drama', **dict):
    '''
    Esta función realiza una llamada a una API a partir de una url y guarda la información en un diccionario.

    Parametros:
    -----------
    url (str): la dirección url de la API a la que queremos hacer la llamada
    type (str): El tipo de pelícua que queremos buscar. By default = movie
    genere (dtr): El género que ueremos buscar. By default = Drama
    dict (dict): el diccionario dónde se almacenará la información

    Returns:
    -------
    dict (dict): el diccionario donde se almacenará la información
    '''
    #definimos la querystring y la API key
    querystring = {"titleType":type,"startYear":"2010","endYear":"2024","genre":genere}

    API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
    headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}    

    #iniciamos el loop while para navegar por las distintas páginas
    while url.startswith('https'):

        #realizmos la llamada y almacenamos los códigos de respuesta en la lista de respuestas
        response_pelis = requests.get(url, headers=headers, params=querystring)
        lista_respuestas.append(response_pelis.status_code)

        if response_pelis.status_code == 200:

            #convertimos los resulatos a formato json
            js_response_pelis = response_pelis.json()

            #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
            next = js_response_pelis['next']

            #guardamos la parte donde tenemos la infomación en una variable
            info_pelis = js_response_pelis['results']

            #guardamos toda la información en el diccionario con un for loop    
            for peli in info_pelis:

                dict['Tipo'].append(peli['titleType']['text'])
                dict['Nombre'].append(peli['titleText']['text'])
                dict['Id'].append(peli['id'])
                dict['Año_estreno'].append(peli['releaseYear']['year'])

                #Hacemos un try/expect para las películas que no se han estrenado todavía            
                try:
                    dict['Mes_estreno'].append(peli['releaseDate']['month'])
                except:            
                    dict['Mes_estreno'].append('por estrenar')

            #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
            try:
                #definimos la base de la nueva url
                url_fixed = "https://moviesdatabase.p.rapidapi.com"
                #definimos la nueva url
                url=url_fixed+next
                #definimos el nuevo querystring
                querystring = {}

            except:
                break

        else:
            print(f'Error {response_pelis.status_code} en la llamada a la API')
            break

    print(f'Fin: Se han añadido l@s {type}s del género {genere} a tu diccionario.')

In [24]:
# Iniciamos el loop for para conseguir información de todos los tipos y géneros
for type in titleType_list:
    for genere in genere_list:
        llamar_API(url,type,genere, **dic_API_Pelis)

Error 429 en la llamada a la API
Fin: Se han añadido l@s movies del género Drama a tu diccionario.
Error 429 en la llamada a la API
Fin: Se han añadido l@s movies del género Action a tu diccionario.
Error 429 en la llamada a la API
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Error 429 en la llamada a la API
Fin: Se han añadido l@s shorts del género Drama a tu diccionario.
Error 429 en la llamada a la API
Fin: Se han añadido l@s shorts del género Action a tu diccionario.
Error 429 en la llamada a la API
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.


In [25]:
#Almacenamos la infomacion del diccionario en distintas listas, cada una corresponde a una key 
lista_tipos = dic_API_Pelis['Tipo']
lista_nombre = dic_API_Pelis['Nombre']
lista_anno = dic_API_Pelis['Año_estreno']
lista_mes = dic_API_Pelis['Mes_estreno']
lista_id = dic_API_Pelis['Id']

In [26]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_pelis = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id)
lista_tuplas_pelis = list(zip_pelis)
lista_tuplas_pelis

[('Movie', "Mom's Coming", 2024, 11, 'tt10579986'),
 ('Movie', 'Love Song and Power', 2024, 1, 'tt10597228'),
 ('Movie', 'Spaceman', 2024, 3, 'tt11097384'),
 ('Movie', 'If - Låtsaskompisar', 2024, 5, 'tt11152168'),
 ('Movie', 'The Accompanist Awakening', 2024, 12, 'tt11237950'),
 ('Movie', 'Manje Bistre 3', 2024, 7, 'tt11822244'),
 ('Movie', 'Eternal Oath', 2024, 7, 'tt12053936'),
 ('Movie', 'The Passion of the Gods', 2024, 6, 'tt12193500'),
 ('Movie', 'Escaping Tomorrow', 2024, None, 'tt12255500'),
 ('Movie', 'After dark', 2024, 12, 'tt12767592'),
 ('Movie', 'Heaven and Hell', 2024, 7, 'tt12861438'),
 ('Movie', 'Broken Glass', 2024, 2, 'tt13036100'),
 ('Movie', 'A Quiet Place: Day One', 2024, 6, 'tt13433802'),
 ('Movie', 'Three Couples Freed', 2024, 1, 'tt13561128'),
 ('Movie', 'Vo Ladki', 2024, 11, 'tt13576680'),
 ('Movie', 'Gina', 2024, 9, 'tt1365464'),
 ('Movie', "We Don't Say Retard Anymore", 2024, 4, 'tt13695406'),
 ('Movie', 'Reckless', 2024, 11, 'tt13965596'),
 ('Movie', 'An Ir